# Grids: Uniformly-Spaced Cartesian Grids

Grids are a datastructure that represent one or more physical quantities that share spatial coordiantes. For example, the density or magnetic field in a plasma as specified on a Cartesian grid. In addition to storing data, grids have built-in interpolator functions for estimating the values of quantities in between grid vertices.

## Creating a grid

In [18]:
%matplotlib inline

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from plasmapy.plasma import grids

A grid can be created either by providing three arrays of spatial coordinates for vertices (eg. x,yz positions) or using a np.linspace-like syntax. For example, the two following methods are equivalent:

In [19]:
#Method 1
xaxis,yaxis, zaxis = [np.linspace(-1*u.cm, 1*u.cm, num=20)]*3
x,y,z = np.meshgrid(xaxis, yaxis, zaxis, indexing='ij')
grid = grids.CartesianGrid(x,y,z)

#Method 2
grid = grids.CartesianGrid(np.array([-1,-1,-1])*u.cm, np.array([1,1,1])*u.cm, num=(150,150,150))

The grid object provides access to a number of properties

In [20]:
print(f"Is the grid uniformly spaced? {grid.is_uniform_grid}")
print(f"Grid shape: {grid.shape}")
print(f"Grid units: {grid.units}")
print(f"Grid spacing on xaxis: {grid.dax0:.2f}")

Is the grid uniformly spaced? True
Grid shape: (150, 150, 150)
Grid units: [Unit("cm"), Unit("cm"), Unit("cm")]
Grid spacing on xaxis: 0.01 cm


The grid points themselves can be accessed in one of two forms

In [21]:
x,y,z = grid.grids
x.shape

(150, 150, 150)

In [22]:
xyz = grid.grid
xyz.shape

(150, 150, 150, 3)

And the axes can be accessed similarly. 

In [23]:
xaxis = grid.ax0
xaxis.shape

(150,)

## Adding Quantities

Now that the grid has been initialized, we can add quantities to it that represent physical properties defined on the grid verticies. Each quantity must be a u.Quantity arry of the same shape as the grid.

In [24]:
Ex = np.random.rand(*grid.shape)*u.V/u.m
Ey = np.random.rand(*grid.shape)*u.V/u.m
Ez = np.random.rand(*grid.shape)*u.V/u.m
Bz = np.random.rand(*grid.shape)*u.T
Bz.shape

(150, 150, 150)

When quantities are added to the grid, they are associated with a key string (just like a dictionary). Any key string can be used, but PlasmaPy functions use a shared set of recognized keys to automatically interperet quantities. The full list of recognized keys can be accessed in the module:

In [25]:
for key in grids.recognized_keys.keys():
    name,unit = grids.recognized_keys[key]
    print(f"{key} -> {name} ({unit})")

x -> x spatial position (m)
y -> y spatial position (m)
z -> z spatial position (m)
rho -> Mass density (kg / m3)
E_x -> Electric field (x component) (V / m)
E_y -> Electric field (y component) (V / m)
E_z -> Electric field (z component) (V / m)
B_x -> Magnetic field (x component) (T)
B_y -> Magnetic field (y component) (T)
B_z -> Magnetic field (z component) (T)


Quantities can be added to the grid one at a time, or in groups

In [26]:
grid.add_quantity('B_z', Bz)
grid.add_quantities(['E_x', 'E_y', 'E_z'], [Ex, Ey, Ez])

Adding an unrecognized quantity will lead to a warning

In [27]:
custom_quantity = np.random.rand(*grid.shape)*u.T*u.mm
grid.add_quantity("int_B", custom_quantity)

C:\Users\Peter\Anaconda3\envs\plasmapy\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Warning: int_B is not recognized quantity key
  


A summary of the grid, including the currently-defined quantities, can be produced by printing the grid object

In [28]:
print(grid)

*** Grid summary ***:
<class 'plasmapy.plasma.grids.CartesianGrid'>
Shape: (150, 150, 150)
Units: [Unit("cm"), Unit("cm"), Unit("cm")]
Uniformly Spaced, dx,dy,dz = (0.013 cm,0.013 cm,0.013 cm)
-----------------------------
Recognized Quantities:
-> B_z (T)
-> E_x (V / m)
-> E_y (V / m)
-> E_z (V / m)
-----------------------------
Unrecognized Quantities:
-> int_B (mm T)



## Interpolating Quantities

Grid objects contain several interpolator methods to evaluate quantites at positions between the grid verticies. These interpolators use the fact that the quantities are defined on the same grid to perform faster interpolations using a nearest-neighbor scheme. When an interpolation at a position is requested, the grid indices closest to that position are calculated. Then, the quantity arrays are evaluated at the interpolated indices. Using this method, many quantities can be interpolated at the same positions in almost the same amount of time as is required to interpolate one quantity. 

Positions are provided to the interpolator as an u.Quantity array of shape [N,3] where N is the number of positions and [i,:] represents the x,y,z coordinates of the ith position:

In [29]:
pos = np.array([[0.1, -0.3, 0], [0.5, .25, .8]]) * u.cm
print(f"Pos shape: {pos.shape}")
print(f"Position 1: {pos[0,:]}")
print(f"Position 2: {pos[1,:]}")

Pos shape: (2, 3)
Position 1: [ 0.1 -0.3  0. ] cm
Position 2: [0.5  0.25 0.8 ] cm


The simplest interpolator directly returns the nearest-neighbor values for each quantity. Positions that are out-of-bounds return an interpolated value of zero.

In [30]:
Ex_vals = grid.nearest_neighbor_interpolator(pos, "E_x")
print(f"Ex at position 1: {Ex_vals[0]:.2f}")

Ex at position 1: 0.63 V / m


Multiple values can be interpolated at the same time by including additional keys as arguments. In this case, the interpolator returns a tuple of arrays as a result.

In [31]:
Ex_vals, Ey_vals, Ez_vals = grid.nearest_neighbor_interpolator(pos, "E_x","E_y","E_z")
print(f"E at position 1: ({Ex_vals[0]:.2f},{Ey_vals[0]:.2f},{Ez_vals[0]:.2f})")

E at position 1: (0.63 V / m,0.39 V / m,0.33 V / m)


For a higher-order interpolation, some grids (such as the CartesianGrid subclass) also include a volume-weighted interpolator. This interpolator averages the values on the eight grid verticies surrounding the position (weighted by their distance). The syntax for using this interpolator is the same:

In [32]:
Ex_vals, Ey_vals, Ez_vals = grid.volume_averaged_interpolator(pos, "E_x","E_y","E_z")
print(f"E at position 1: ({Ex_vals[0]:.2f},{Ey_vals[0]:.2f},{Ez_vals[0]:.2f})")

E at position 1: (0.59 V / m,0.48 V / m,0.42 V / m)
